## Introduction

The Azure AI Agent Service Code Interpreter enables the LLM to generate Python code for tasks such as creating charts or performing complex data analyses based on user queries. It leverages natural language processing (NLP), sales data from an SQLite database, and user prompts to automate code generation. The LLM-generated Python code executes within a secure sandbox environment, utilizing a restricted subset of Python to ensure safe and controlled execution.


### Lab Exercise

In this lab, you'll enable the Code Interpreter to execute Python code generated by the LLM.

In [ ]:
import os
import sys
from dotenv import load_dotenv


# get current folder
current_directory = os.getcwd()
# get parent folder
parent_directory = os.path.abspath(os.path.join(current_directory, ".."))
sys.path.append(parent_directory)

project_directory = os.path.abspath(os.path.join(parent_directory, ".."))
sys.path.append(project_directory)

load_dotenv('.env')

In [ ]:
from main import initialize
from azure.ai.projects.aio import AIProjectClient
from azure.ai.projects.models import (
    Agent,
    AgentThread,
    AsyncFunctionTool,
    AsyncToolSet,
    BingGroundingTool,
    CodeInterpreterTool,
    FileSearchTool,
)
from azure.identity import DefaultAzureCredential
from azure.monitor.opentelemetry import configure_azure_monitor
from opentelemetry import trace
from sales_data import SalesData
from stream_event_handler import StreamEventHandler
from terminal_colors import TerminalColors as tc
from utilities import Utilities


In [ ]:
TENTS_DATA_SHEET_FILE = "/home/azureuser/azure-ai-agent-workshop/azure-ai-agent-service-sampleCode/datasheet/contoso-tents-datasheet.pdf"
API_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_DEPLOYMENT")
PROJECT_CONNECTION_STRING = os.environ["PROJECT_CONNECTION_STRING"]
BING_CONNECTION_NAME = os.getenv("BING_CONNECTION_NAME")
MAX_COMPLETION_TOKENS = 4096
MAX_PROMPT_TOKENS = 10240
TEMPERATURE = 0.2

toolset = AsyncToolSet()
sales_data = SalesData()
utilities = Utilities()

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=PROJECT_CONNECTION_STRING,
)

functions = AsyncFunctionTool(
    {
        sales_data.async_fetch_sales_data_using_sqlite_query,
    }
)


### Run the Agent App

Start a Conversation with the Agent

Try these questions:

1. Show sales by region as a pie chart

Once the task is complete, the pie chart image will be saved in the files folder. Open the folder in VS Code and click on the image file to view it.


2. Download the sales data

Once the task is complete, check the files folder to see the downloaded file.

3. Download as JSON

Once the task is complete, check the files folder to see the downloaded file.

4. Continue asking questions about Contoso sales data to see the Code Interpreter in action.

In [7]:
from terminal_colors import TerminalColors as tc
from main import post_message, cleanup

INSTRUCTIONS_FILE = "/home/azureuser/azure-ai-agent-workshop/azure-ai-agent-service-sampleCode/instructions/instructions_code_interpreter.txt"

agent, thread = await initialize()

while True:
    # Get user input prompt in the terminal using a pretty shade of green
    print("\n")
    prompt = input(f"{tc.GREEN}Enter your query (type exit to finish): {tc.RESET}")
    if prompt.lower() == "exit":
        break
    if not prompt:
        continue
    await post_message(agent=agent, thread_id=thread.id, content=prompt, thread=thread)

await cleanup(agent, thread)

Uploading file: /home/azureuser/azure-ai-agent-workshop/azure-ai-agent-service-sampleCode/datasheet/contoso-tents-datasheet.pdf
Creating the vector store
Vector store created and files added.
Creating agent...
Created agent, ID: asst_Ut17IZkrxjPyGioR8kC3uzFg
Creating thread...
Created thread, ID: thread_HUudirJ1i2PDLOhxxN2JapCk



Function Call Tools: async_fetch_sales_data_using_sqlite_query

Executing query: SELECT region, SUM(revenue) AS total_revenue FROM sales_data GROUP BY region

Here is the pie chart showing sales revenue by region. You can download the chart using the link below:

[Download Sales by Region Pie Chart](sandbox:/mnt/data/sales_by_region_pie_chart.png)Getting file with ID: assistant-2uaaWJw5qxU9b8XITR830feJ
File saved to files/sales_by_region_pie_chart.assistant-2uaaWJw5qxU9b8XITR830feJ.png


